## Klassisk mekanikk numerisk øving 3
Eirik Jaccheri Høydalsvik, Hans Gløckner Giil

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#some constants
t0 = 0
g = 9.8
l = g
m = 1
q = 0.5
omega_d = 2/3
omega = np.sqrt(g / l) #NB; we will use omega as a helping constant, and thetaDot as the time-derivative of the angle

theta0 = 0.2
thetaDot0 = 0
u0 = np.array((theta0, thetaDot0))

#RK4, solves differential equation f. here: u0 = [theta, theta_dot]
def kutta_4(f, u0, t0, tNum, dt, q, F_d, omega_d):
    U = np.zeros( ((tNum),  u0.size) )
    U[0] = u0
    for i in range(1 ,tNum):
        F1 = f(i * dt, U[i-1], q, F_d, omega_d)
        F2 = f(i * dt + dt / 2, U[i-1] + dt  / 2 * F1, q, F_d, omega_d)
        F3 = f(i * dt + dt / 2, U[i-1] + dt / 2 * F2, q, F_d, omega_d)
        F4 = f(i * dt + dt, U[i-1] + dt * F3, q, F_d, omega_d)
        U[i] = U[i-1] + dt / 6 * (F1 + 2* F2 + 2 * F3 + F4)        
    return U

def f_driven (t, u, q, F_d , omega_d): #with driving force F(t) / m / l = D_d * sin(omega_d* t) 
    return np.array([u[1], - omega**2 * np.sin(u[0]) - q * u[1]  + F_d * np.sin(omega_d * t)])




In [ ]:
def time_evolve(u0,t0, tNum, dt, q, F_d, omega_d):
    t_arr = np.linspace(t0, t0 + dt * tNum, tNum)
    u_arr = kutta_4(f_driven, u0, t0, tNum, dt, q, F_d, omega_d)
    u_arr = (u_arr + np.pi) % (2*np.pi) - np.pi 
    return t_arr, u_arr

def plot_theta_thetaDot(u0,t0, tNum, dt, q, F_d_arr, omega_d):
    
    fig, (ax1, ax2) = plt.subplots(2, 1)
    fig.suptitle("Figure 1", fontsize = 18,  y = 0)
    fig.set_size_inches(12,10)
    ax1.set_xlabel("Time", fontsize = 18)
    ax1.set_ylabel("Angle", fontsize = 18)
    ax2.set_xlabel("Time", fontsize = 18)
    ax2.set_ylabel("Angular velocity", fontsize = 18)
    
    for F_d in F_d_arr:
        t_arr, u_arr = time_evolve(u0,t0, tNum, dt, q, F_d, omega_d)
        l1, = ax1.plot(t_arr,u_arr.T[0], label = "$F_d$ = " + str(F_d))
        l2, = ax2.plot(t_arr, u_arr.T[1], label = "$F_d$ = " + str(F_d))
    plt.tight_layout()
    ax1.legend(loc = "upper left")
    ax2.legend(loc = "upper left")
    plt.show()
    
def phase_difference(u0,t0, tNum, dt, q, F_d, omega_d, del_theta):
    u0_offset = np.array([u0[0] + del_theta , u0[1]])
    
    t_arr, u_arr = time_evolve(u0,t0, tNum, dt, q, F_d, omega_d)
    t_arr_offset, u_arr_offset = time_evolve(u0_offset,t0, tNum, dt, q, F_d, omega_d)
    
    del_theta_arr = u_arr.T[0] - u_arr_offset.T[0]
    
    return  t_arr_offset, u_arr_offset,  del_theta_arr

def plot_phase_difference(u0,t0, tNum, dt, q, F_d, omega_d, del_theta, label):
    t_arr, u_arr,  del_theta_arr = phase_difference(u0,t0, tNum, dt, q, F_d, omega_d, del_theta)
    
    fig, ax = plt.subplots()
    fig.suptitle(label, y = 0, fontsize = 18)
    fig.set_size_inches(8, 5)
    ax.semilogy(t_arr,del_theta_arr) 
    ax.set_xlabel("Time", fontsize = 18)
    ax.set_ylabel("Angle difference", fontsize = 18)
    plt.tight_layout()
    plt.show()
    
def plot_phase_space(u0,t0, tNum, dt, q, F_d, omega_d):
    t_arr, u_arr = time_evolve(u0,t0, tNum, dt, q, F_d, omega_d)
    
    fig, (ax1,ax2) = plt.subplots(1,2)
    ax1.plot(u_arr.T[0],u_arr.T[1])
    n = int(2*np.pi / dt / omega_d)
    
    ax2.plot(u_arr.T[0][0::n],u_arr.T[1][0::n],".")

## Exercise 1
For the whole excercise we will use the constant values: pendulum length $l = 9.8$, mass $m = 1$, gravitational acceleration $g=9.8$,initial angle $\theta_0 = 0.2$, initial angular velocity $\omega_0 = 0$, damping parameter $q = 0.5$.
Figure 1 shows the angle and the angular velocity for the forced real physical pendulum, using three different values for the driving force $F_d$. The angle is shown in the upper plot, and the angular velocity is shown in the lower plot.  

In [ ]:
dt = 0.05
tNum = int(60 / dt)
F_d_arr = np.array([0.2, 0.5, 1.2])
plot_theta_thetaDot(u0,t0, tNum, dt, q, F_d_arr, omega_d)

## Exercise 2
Figure 3 and 4 shows the angle difference between the solution from exercise 1, and the solution with initial angle displacement $ \theta_0 + 0.001$ and zero initial angular velocity, plotted against time. Figure 3 shows the case where $F_d = 0.5$, and figure 4 shows  the case where $F_d = 1.2$. For $F_d = 0.5$, the angle difference decreases with a linear trend with time, and for the case $F_d$ the angle difference increases with a linear tendency with time. The pendulum in Figure 3 shows normal behavior, and Figure 4 shows a chaotic behavior.

In [ ]:
dt = 0.05
tNum = int(60 / dt)
del_theta = 0.001

F_d = 0.5
plot_phase_difference(u0,t0, tNum, dt, q, F_d, omega_d, del_theta, "Figure 2")

F_d = 1.2
plot_phase_difference(u0,t0, tNum, dt, q, F_d, omega_d, del_theta, "Figure 3")

In [ ]:
dt = np.pi / 1000 / omega_d
tNum = int(2000/dt)
F_d = 1.2
plot_phase_space(u0,t0, tNum, dt, q, F_d, omega_d)